<a href="https://colab.research.google.com/github/circuit-geek/Summer-Internship-2021/blob/master/Basic_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Introduction to Neural Networks with PyTorch**

In [1]:
import torch
import torch.nn as nn

In [5]:
inputs = torch.Tensor([[73., 67., 43.],
                       [91., 88., 64.],
                       [87., 134., 58.],
                       [102., 43., 37.],
                       [69., 96., 70.],
                       [74., 66., 43.],
                       [91., 87., 65.],
                       [88., 134., 59.],
                       [101., 44., 37.],
                       [68., 96., 71.],
                       [73., 66., 44.],
                       [92., 87., 64.],
                       [87., 135., 57.],
                       [103., 43.,36.],
                       [68., 97., 70.]])

targets = torch.Tensor([[56. , 70.],
                        [81., 101.],
                        [119., 133.],
                        [22.,37.],
                        [103., 119.],
                        [57.,69.],
                        [80.,102.],
                        [118., 132.],
                        [21.,38.],
                        [104.,118.],
                        [57.,69.],
                        [82.,100.],
                        [118.,134.],
                        [20.,38.],
                        [102., 120.]])

print(inputs.dtype, targets.dtype)


torch.float32 torch.float32


Unlike in the above example, where we have 15 inputs and target values, in real life dataset we will have 1000's values and many rows, so we break down these into several batches and perform gradient descent on them. So for this we use Dataset and Dataloader from torch utilities.

In [3]:
from torch.utils.data import TensorDataset

In [6]:
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

Now that we have prepared the dataset, now we use the dataloader to actually separate the data into batches again by using torch utilities, we have define the batch size while doing so.

In [7]:
from torch.utils.data import DataLoader

In [8]:
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [9]:
for xd, yd in train_dl:
  print(xd)
  print(yd)
  break

tensor([[ 87., 134.,  58.],
        [101.,  44.,  37.],
        [ 69.,  96.,  70.],
        [ 87., 135.,  57.],
        [ 73.,  67.,  43.]])
tensor([[119., 133.],
        [ 21.,  38.],
        [103., 119.],
        [118., 134.],
        [ 56.,  70.]])


In [10]:
model = nn.Linear(3 ,2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.3100, -0.5393,  0.4733],
        [-0.3173,  0.5313, -0.5022]], requires_grad=True)
Parameter containing:
tensor([-0.4349,  0.2861], requires_grad=True)


In [12]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.3100, -0.5393,  0.4733],
         [-0.3173,  0.5313, -0.5022]], requires_grad=True),
 Parameter containing:
 tensor([-0.4349,  0.2861], requires_grad=True)]

parameters() method returns a list of all the weights and bias matrices present in the model, for this linear regression, it returns one weight matrix and one bias matrix.

In [13]:
preds = model(inputs)
print(preds)

tensor([[  6.4180,  -8.8712],
        [ 10.6132, -13.9705],
        [-18.2736,  14.7504],
        [ 25.5118, -27.8093],
        [  2.3181,  -5.7536],
        [  7.2673,  -9.7197],
        [ 11.6258, -15.0039],
        [-17.4903,  13.9309],
        [ 24.6625, -26.9608],
        [  2.4814,  -5.9385],
        [  7.4306,  -9.9047],
        [ 11.4625, -14.8190],
        [-19.2862,  15.7838],
        [ 25.3485, -27.6244],
        [  1.4688,  -4.9051]], grad_fn=<AddmmBackward>)


In [14]:
import torch.nn.functional as F

In [15]:
loss_fn = F.mse_loss

loss = loss_fn(model(inputs), targets)
print(loss)

tensor(8951.0713, grad_fn=<MseLossBackward>)


In [16]:
opt = torch.optim.SGD(model.parameters(), lr = 1e-5)

nn.functional has all the important functions like loss functions, activation functions that are needed for neural networks, and here we are SGD optimizer which taken in model.parameter() and the learning as the arguments. 
Now we are ready to train the model.


1.   Generate Predictions
2.   Calculate Loss
3. Compute Gradients
4. Update the parameters using gradients
5. Reset the gradients to zero.




In [18]:
def fit(epochs, model, loss_fn, opt, train_dl):
  for epoch in range(epochs):
    for xd, yd in train_dl:
      pred = model(xd)
      loss = loss_fn(pred, yd)
      loss.backward()
      opt.step() # automatically update the gradients.
      opt.zero_grad()
    if (epoch+1) % 10 == 0:
      print('Epoch[{}/{}], Loss: {:.4f}'.format(epoch+1, epochs, loss.item())) 

In [19]:
fit(100, model, loss_fn, opt, train_dl)

Epoch[10/100], Loss: 631.9855
Epoch[20/100], Loss: 389.4392
Epoch[30/100], Loss: 282.2800
Epoch[40/100], Loss: 277.1795
Epoch[50/100], Loss: 177.5627
Epoch[60/100], Loss: 115.9147
Epoch[70/100], Loss: 96.5891
Epoch[80/100], Loss: 99.8172
Epoch[90/100], Loss: 62.0755
Epoch[100/100], Loss: 54.4914


In [21]:
new_preds = model(inputs)
print(new_preds)

tensor([[ 58.5343,  71.4691],
        [ 84.1839,  94.9368],
        [111.3209, 144.2365],
        [ 30.1055,  42.7361],
        [100.0561, 105.8753],
        [ 57.6002,  70.3139],
        [ 84.2956,  93.9625],
        [111.8357, 144.2644],
        [ 31.0396,  43.8913],
        [101.1020, 106.0561],
        [ 58.6460,  70.4948],
        [ 83.2498,  93.7816],
        [111.2091, 145.2108],
        [ 29.0596,  42.5553],
        [100.9902, 107.0304]], grad_fn=<AddmmBackward>)


In [22]:
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])
